# Proteomics

# Libraries to Import

Be sure to activate the "biopython" conda environment.

In [ ]:
# !jupyter-nbextension enable nglview --py --sys-prefix

In [3]:
from Bio.PDB import *
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import nglview as nv
import ipywidgets

import warnings
warnings.filterwarnings('ignore')

# Using the PDB (Protein Data Bank) File Format

In [2]:
parser = PDBParser()

In [3]:
structure = parser.get_structure("PHA-L", "Data/1FAT.pdb")

In [4]:
view = nv.show_biopython(structure)
view

NGLWidget()

# Using the CIF (Crystallographic Information File) File Format

In [4]:
parser = MMCIFParser()

In [5]:
structure = parser.get_structure("PHA-L", "fa/1fat.cif")

In [6]:
view = nv.show_biopython(structure)
view

NGLWidget()

# Diving into the Header Information

In [7]:
mmcif_dict = MMCIF2Dict.MMCIF2Dict("fa/1fat.cif")

In [8]:
# list(mmcif_dict.keys())

**What’s the overall layout of a Structure object?**

The Structure object follows the so-called SMCRA (Structure/Model/Chain/Residue/Atom) architecture :

- A structure consists of models
- A model consists of chains
- A chain consists of residues
- A residue consists of atoms

**Accessing Residue Sequence**

The above molecule has 4 major models, which is represented below when resseq (Residue Sequence) resets for each model.

In [9]:
# Iterate over all residues in a model
# for model in structure:
#     for residue in model.get_residues():
#         print(residue)

In [10]:
residues = structure.get_residues() # returns a generator object

In [11]:
# [item for item in residues]

In [12]:
res_list = Selection.unfold_entities(structure, "R")

**Polypeptide Builder**

Peptides are characterized by start codons, a sequence of length N, and a stop codon, hence why 4 models can be comprised of 7 chains. You can see the 4 major chains comprising the above structure as well as 3 linking chains.

In [13]:
# Using CA-CA
ppb = CaPPBuilder()
counter = 1
for pp in ppb.build_peptides(structure):
    print("Sequence: ", counter)
    print(pp.get_sequence())
    counter += 1

Sequence:  1
SNDIYFNFQRFNETNLILQRDASVSSSGQLRLTNLN
Sequence:  2
NGEPRVGSLGRAFYSAPIQIWDNTTGTVASFATSFTFNIQVPNNAGPADGLAFALVPVGSQPKDKGGFLGLFDGSNSNFHTVAVEFDTLYNKDWDPTERHIGIDVNSIRSIKTTRWDFVNGENAEVLITYDSSTNLLVASLVYPSQKTSFIVSDTVDLKSVLPEWVSVGFSATTGINKGNVETNDVLSWSFASKLS
Sequence:  3
SNDIYFNFQRFNETNLILQRDASVSSSGQLRLTNLNGNGEPRVGSLGRAFYSAPIQIWDNTTGTVASFATSFTFNIQVPNNAGPADGLAFALVPVGSQPKDKGGFLGLFDGSNSNFHTVAVEFDTLYNKDWDPTERHIGIDVNSIRSIKTTRWDFVNGENAEVLITYDSSTNLLVASLVYPSQKTSFIVSDTVDLKSVLPEWVSVGFSATTGINKGNVETNDVLSWSFASKLS
Sequence:  4
SNDIYFNFQRFNETNLILQRDASVSSSGQLRLTNLN
Sequence:  5
NGEPRVGSLGRAFYSAPIQIWDNTTGTVASFATSFTFNIQVPNNAGPADGLAFALVPVGSQPKDKGGFLGLFDGSNSNFHTVAVEFDTLYNKDWDPTERHIGIDVNSIRSIKTTRWDFVNGENAEVLITYDSSTNLLVASLVYPSQKTSFIVSDTVDLKSVLPEWVSVGFSATTGINKGNVETNDVLSWSFASKLS
Sequence:  6
SNDIYFNFQRFNETNLILQRDASVSSSGQLRLTNL
Sequence:  7
NGEPRVGSLGRAFYSAPIQIWDNTTGTVASFATSFTFNIQVPNNAGPADGLAFALVPVGSQPKDKGGFLGLFDGSNSNFHTVAVEFDTLYNKDWDPTERHIGIDVNSIRSIKTTRWDFVNGENAEVLITYDSSTNLLVASLVYPSQKTSFIVSDTVDLKSVLPEWVSVGFS

With our different chains, we can run protein analysis methods from ProteinAnalysis and store the results in a dict.

Protein scale analysis is omitted due to the number of scales that can be examined for. One should run scale analysis ad hoc as detailed below.

In [14]:
# Define parser and get structure from file
parser = MMCIFParser()
structure = parser.get_structure("PHA-L", "fa/1fat.cif")

# Define polypeptide builder
ppb = CaPPBuilder()

# Create empty list for chains
all_seqs = []
counter = 1

# For each polypeptide in the structure, run protein analysis methods and store in dict
for pp in ppb.build_peptides(structure):
    seq_info = {}
    seq = pp.get_sequence()
    analyzed_seq = ProteinAnalysis(str(seq)) # needs to be a str

    seq_info['Sequence Number'] = counter # set sequence id
    seq_info['Sequence'] = seq # store Seq() object
    seq_info['Sequence Length'] = len(seq) # length of seq
    seq_info['Molecular Weight'] = round(analyzed_seq.molecular_weight(), 2) # mol weight
    seq_info['GRAVY'] = round(analyzed_seq.gravy(), 4) # average hydrophobicity
    seq_info['Amino Acid Count'] = analyzed_seq.count_amino_acids() # count residues
    seq_info['Amino Acid Percent'] = analyzed_seq.get_amino_acids_percent() # normalized count
    seq_info['Secondary Structure'] = analyzed_seq.secondary_structure_fraction() # helix, turn, sheet
    
    # Update all_seqs list and increase counter
    all_seqs.append(seq_info)
    counter += 1

Now we have a list of dicts that can be indexed to select information easily. 

In [15]:
all_seqs[0]

{'Sequence Number': 1,
 'Sequence': Seq('SNDIYFNFQRFNETNLILQRDASVSSSGQLRLTNLN'),
 'Sequence Length': 36,
 'Molecular Weight': 4176.52,
 'GRAVY': -0.5611,
 'Amino Acid Count': {'A': 1,
  'C': 0,
  'D': 2,
  'E': 1,
  'F': 3,
  'G': 1,
  'H': 0,
  'I': 2,
  'K': 0,
  'L': 5,
  'M': 0,
  'N': 6,
  'P': 0,
  'Q': 3,
  'R': 3,
  'S': 5,
  'T': 2,
  'V': 1,
  'W': 0,
  'Y': 1},
 'Amino Acid Percent': {'A': 0.027777777777777776,
  'C': 0.0,
  'D': 0.05555555555555555,
  'E': 0.027777777777777776,
  'F': 0.08333333333333333,
  'G': 0.027777777777777776,
  'H': 0.0,
  'I': 0.05555555555555555,
  'K': 0.0,
  'L': 0.1388888888888889,
  'M': 0.0,
  'N': 0.16666666666666666,
  'P': 0.0,
  'Q': 0.08333333333333333,
  'R': 0.08333333333333333,
  'S': 0.1388888888888889,
  'T': 0.05555555555555555,
  'V': 0.027777777777777776,
  'W': 0.0,
  'Y': 0.027777777777777776},
 'Secondary Structure': (0.3333333333333333,
  0.3333333333333333,
  0.19444444444444445)}

We can also perform protein analysis ad-hoc from stored sequences in the dict.

In [16]:
seq1 = all_seqs[0]['Sequence']

A note here though is that ProteinAnalysis() requires a string of the sequence, not a Seq() object. Biopython has an overloaded str() method that can retrieve the raw string from the Seq() object.  

In [17]:
analysed_seq = ProteinAnalysis(str(seq1))

**Molecular Weight**

In [18]:
analysed_seq.molecular_weight()

4176.516699999999

**Gravy**

> Protein GRAVY returns the GRAVY (grand average of hydropathy) value for the protein sequences you enter. The GRAVY value is calculated by adding the hydropathy value for each residue and dividing by the length of the sequence (Kyte and Doolittle; 1982).

A higher value is increased hydrophobicity.

[Source](https://pubmed.ncbi.nlm.nih.gov/7108955/):
Kyte J, Doolittle RF (May 1983). "A simple method for displaying the hydropathic character of a protein". J. Mol. Biol. 157 (1): 105–32. PMID 7108955

In [19]:
analysed_seq.gravy()

-0.561111111111111

**Amino Acid Count**

In [20]:
analysed_seq.count_amino_acids()

{'A': 1,
 'C': 0,
 'D': 2,
 'E': 1,
 'F': 3,
 'G': 1,
 'H': 0,
 'I': 2,
 'K': 0,
 'L': 5,
 'M': 0,
 'N': 6,
 'P': 0,
 'Q': 3,
 'R': 3,
 'S': 5,
 'T': 2,
 'V': 1,
 'W': 0,
 'Y': 1}

**Amino Acid Percentage**

In [21]:
analysed_seq.get_amino_acids_percent()

{'A': 0.027777777777777776,
 'C': 0.0,
 'D': 0.05555555555555555,
 'E': 0.027777777777777776,
 'F': 0.08333333333333333,
 'G': 0.027777777777777776,
 'H': 0.0,
 'I': 0.05555555555555555,
 'K': 0.0,
 'L': 0.1388888888888889,
 'M': 0.0,
 'N': 0.16666666666666666,
 'P': 0.0,
 'Q': 0.08333333333333333,
 'R': 0.08333333333333333,
 'S': 0.1388888888888889,
 'T': 0.05555555555555555,
 'V': 0.027777777777777776,
 'W': 0.0,
 'Y': 0.027777777777777776}

**Secondary Structure**

Returns a tuple of (helix, turn, sheet) percentage. Note that not all residues belong to a secondary structure, hence why the sum(fractions) != 1

In [22]:
analysed_seq.secondary_structure_fraction() # helix, turn, sheet

(0.3333333333333333, 0.3333333333333333, 0.19444444444444445)

**Protein Scales**

Scales are located [here](https://github.com/biopython/biopython/blob/master/Bio/SeqUtils/ProtParamData.py#L6).

- Kyte & Doolittle index of hydrophobicity --> kd
- Normalized flexibility parameters (B-values), average --> flex
- Hydrophilicity --> hw
- Surface accessibility --> em
- Janin Interior to surface transfer energy scale --> ja

In [23]:
kd = {"A": 1.8, "R": -4.5, "N": -3.5, "D": -3.5, "C": 2.5,
      "Q": -3.5, "E": -3.5, "G": -0.4, "H": -3.2, "I": 4.5,
      "L": 3.8, "K": -3.9, "M": 1.9, "F": 2.8, "P": -1.6,
      "S": -0.8, "T": -0.7, "W": -0.9, "Y": -1.3, "V": 4.2}

flex = {"A": 0.984, "C": 0.906, "E": 1.094, "D": 1.068,
        "G": 1.031, "F": 0.915, "I": 0.927, "H": 0.950,
        "K": 1.102, "M": 0.952, "L": 0.935, "N": 1.048,
        "Q": 1.037, "P": 1.049, "S": 1.046, "R": 1.008,
        "T": 0.997, "W": 0.904, "V": 0.931, "Y": 0.929}

hw = {"A": -0.5, "R": 3.0, "N": 0.2, "D": 3.0, "C": -1.0,
      "Q": 0.2, "E": 3.0, "G": 0.0, "H": -0.5, "I": -1.8,
      "L": -1.8, "K": 3.0, "M": -1.3, "F": -2.5, "P": 0.0,
      "S": 0.3, "T": -0.4, "W": -3.4, "Y": -2.3, "V": -1.5}

em = {"A": 0.815, "R": 1.475, "N": 1.296, "D": 1.283, "C": 0.394,
      "Q": 1.348, "E": 1.445, "G": 0.714, "H": 1.180, "I": 0.603,
      "L": 0.603, "K": 1.545, "M": 0.714, "F": 0.695, "P": 1.236,
      "S": 1.115, "T": 1.184, "W": 0.808, "Y": 1.089, "V": 0.606}

ja = {"A": 0.28, "R": -1.14, "N": -0.55, "D": -0.52, "C": 0.97,
      "Q": -0.69, "E": -1.01, "G": 0.43, "H": -0.31, "I": 0.60,
      "L": 0.60, "K": -1.62, "M": 0.43, "F": 0.46, "P": -0.42,
      "S": -0.19, "T": -0.32, "W": 0.29, "Y": -0.15, "V": 0.60}

In [24]:
analysed_seq.protein_scale(window=3, param_dict=kd)

[-2.6,
 -0.8333333333333334,
 -0.09999999999999994,
 2.0,
 -0.6666666666666666,
 0.6999999999999998,
 -1.4000000000000001,
 -1.7333333333333334,
 -1.7333333333333334,
 -1.7333333333333334,
 -1.4000000000000001,
 -2.566666666666667,
 -2.566666666666667,
 -0.13333333333333344,
 1.5999999999999999,
 4.033333333333333,
 1.5999999999999999,
 -1.4000000000000001,
 -3.8333333333333335,
 -2.066666666666667,
 -0.8333333333333334,
 1.7333333333333334,
 0.8666666666666667,
 0.8666666666666668,
 -0.8000000000000002,
 -0.6666666666666666,
 -1.5666666666666667,
 -0.03333333333333336,
 -1.4000000000000001,
 1.0333333333333332,
 -0.4666666666666668,
 -0.1333333333333334,
 -0.13333333333333344,
 -1.0666666666666667]